In [1]:
import os
os.environ["D123_DEVKIT_ROOT"] = "/cpfs01/shared/opendrivelab/sii/jingchanghui/d123_workspace/d123"
os.environ["D123_DATA_ROOT"] = "/cpfs01/shared/opendrivelab/sii/jingchanghui/d123_workspace/data"
os.environ["D123_MAPS_ROOT"] = "/cpfs01/shared/opendrivelab/sii/jingchanghui/d123_workspace/data/maps"
os.environ["D123_EXP_ROOT"] = "/cpfs01/shared/opendrivelab/sii/jingchanghui/d123_workspace/exp"
os.environ["NUPLAN_DATA_ROOT"] = "/nas/shared/opendrivelab/nuplan/dataset"
os.environ["NUPLAN_MAPS_ROOT"] = "/nas/shared/opendrivelab/nuplan/dataset/maps"
os.environ["PYTHONPATH"] = "/cpfs01/shared/opendrivelab/sii/jingchanghui/d123_workspace/d123/tool/nuscenes-devkit/python-sdk:" + os.environ.get("PYTHONPATH", "")
os.environ["NUSCENES_DATA_ROOT"] = "/cpfs01/shared/opendrivelab/datasets/nuscenes"
os.environ["NUSCENES_MAPS_ROOT"] = "/cpfs01/shared/opendrivelab/datasets/nuscenes"
os.environ["CARLA_DATA_ROOT"] = "/nas/shared/opendrivelab/nuplan/dataset"

In [3]:
# 导入库
import lanelet2anchors

# 查看可用功能（如果库支持）
print(dir(lanelet2anchors))

# 尝试使用核心类或函数（示例，需根据实际情况调整）
# anchors = lanelet2anchors.generate_anchors(...)
# result = lanelet2anchors.match_anchors(anchors, lanelets)

['Anchor', 'AnchorGenerator', 'Vehicle', 'VehicleAhead', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'anchor2linestring', 'anchor2polygon', 'anchor_generation', 'anchor_generator', 'anchor_tools', 'find_vehicle_ahead']


In [2]:
from d123.common.visualization.viser.utils import get_map_meshes
from d123.dataset.scene.arrow_scene import ArrowScene
from pathlib import Path

# 1. 先加载一个 NuScenes 或 NuPlan 的 arrow scene
# 这里路径替换成你实际 arrow 文件的路径
scene_path = Path("/cpfs01/shared/opendrivelab/sii/jingchanghui/d123_workspace/data/train/9a61a88ed9094334a73aa93c08222110.arrow")
scene = ArrowScene(scene_path)

# 2. 调用 get_map_meshes
meshes = get_map_meshes(scene)

# 3. 打印 key 和数量
print("返回类型:", type(meshes))
print("mesh dict 长度:", len(meshes))


# 如果是 dict，遍历 key
for key, mesh in meshes.items():
    print(f"layer: {key}, mesh type: {type(mesh)}, vertices: {len(mesh.vertices) if hasattr(mesh, 'vertices') else 'N/A'}")

/cpfs01/shared/opendrivelab/sii/jingchanghui/d123_workspace/d123/d123/dataset/maps/gpkg/gpkg_map.py:76: UserWarning: GPKGMap: stop_line not available in /cpfs01/shared/opendrivelab/sii/jingchanghui/d123_workspace/data/maps/nuscenes_singapore-onenorth.gpkg
  warnings.warn(f"GPKGMap: {map_layer_name} not available in {str(self._file_path)}")
/cpfs01/shared/opendrivelab/sii/jingchanghui/d123_workspace/d123/d123/dataset/maps/gpkg/gpkg_map.py:76: UserWarning: GPKGMap: road_edge not available in /cpfs01/shared/opendrivelab/sii/jingchanghui/d123_workspace/data/maps/nuscenes_singapore-onenorth.gpkg
  warnings.warn(f"GPKGMap: {map_layer_name} not available in {str(self._file_path)}")
/cpfs01/shared/opendrivelab/sii/jingchanghui/d123_workspace/d123/d123/dataset/maps/gpkg/gpkg_map.py:76: UserWarning: GPKGMap: road_line not available in /cpfs01/shared/opendrivelab/sii/jingchanghui/d123_workspace/data/maps/nuscenes_singapore-onenorth.gpkg
  warnings.warn(f"GPKGMap: {map_layer_name} not available in

dict_keys([<MapLayer.LANE_GROUP: 1>, <MapLayer.LANE: 0>, <MapLayer.WALKWAY: 4>, <MapLayer.CROSSWALK: 3>, <MapLayer.CARPARK: 5>, <MapLayer.GENERIC_DRIVABLE: 6>])
返回类型: <class 'dict'>
mesh dict 长度: 6
layer: lane_group, mesh type: <class 'trimesh.base.Trimesh'>, vertices: 14449
layer: lane, mesh type: <class 'trimesh.base.Trimesh'>, vertices: 11885
layer: walkway, mesh type: <class 'trimesh.base.Trimesh'>, vertices: 10573
layer: crosswalk, mesh type: <class 'trimesh.base.Trimesh'>, vertices: 432
layer: carpark, mesh type: <class 'trimesh.base.Trimesh'>, vertices: 147
layer: generic_drivable, mesh type: <class 'trimesh.base.Trimesh'>, vertices: 26334


In [6]:
from pathlib import Path
from d123.dataset.scene.arrow_scene import ArrowScene
from shapely import geometry as geom
import numpy as np

# 1. 加载 ArrowScene
scene_path = Path("/cpfs01/shared/opendrivelab/sii/jingchanghui/d123_workspace/data/train/9a61a88ed9094334a73aa93c08222110.arrow")
scene = ArrowScene(scene_path)

# 2. 获取中心点
initial_ego_state = scene.get_ego_state_at_iteration(0)
center = initial_ego_state.center_se3

# 3. 指定需要的 map layers
# 如果你有 MapLayer 枚举，可以用 MapLayer.LANE.value 等
map_layers = [0, 1, 2, 3, 4, 5, 6]  

# 4. 从 map_api 获取 map objects
MAP_RADIUS = 1000000  # 可调整
map_objects_dict = scene.map_api.get_proximal_map_objects(center.point_2d, radius=MAP_RADIUS, layers=map_layers)

# 5. 定义检查函数
def check_map_surface_outline(map_surface, max_coords=5):
    outline_3d_array = map_surface.outline_3d.array
    print("outline_3d_array type:", type(outline_3d_array))
    print("outline_3d_array shape:", outline_3d_array.shape)

    if outline_3d_array.size == 0:
        print("outline_3d_array is empty!")
        return

    print("sample points:\n", outline_3d_array[:max_coords])

    try:
        poly_2d = geom.Polygon(outline_3d_array[:, :2])
        print("polygon valid:", poly_2d.is_valid)
        print("polygon coords sample:", list(poly_2d.exterior.coords)[:max_coords])
    except Exception as e:
        print("Failed to create Polygon:", e)

# 6. 遍历 map_objects_dict 并调用检查函数
for layer, surfaces in map_objects_dict.items():
    for i, surface in enumerate(surfaces):
        print(f"Layer: {layer}, Surface {i}")
        check_map_surface_outline(surface)
print("map_objects_dict keys:", map_objects_dict.keys())
for layer, surfaces in map_objects_dict.items():
    print(f"Layer {layer}: {len(surfaces)} surfaces")



Layer: 0, Surface 0
outline_3d_array type: <class 'numpy.ndarray'>
outline_3d_array shape: (38, 3)
sample points:
 [[343357.22588298 112764.28031339      0.        ]
 [343349.95991003 112763.28654743      0.        ]
 [343347.0469068  112748.69523883      0.        ]
 [343346.67364417 112747.77181561      0.        ]
 [343345.73442551 112747.38370546      0.        ]]
polygon valid: True
polygon coords sample: [(343357.2258829841, 112764.28031339438), (343349.95991002815, 112763.28654742985), (343347.04690679954, 112748.69523882965), (343346.67364417226, 112747.7718156068), (343345.73442551075, 112747.38370545676)]
Layer: 0, Surface 1
outline_3d_array type: <class 'numpy.ndarray'>
outline_3d_array shape: (24, 3)
sample points:
 [[343380.34185164 112828.06545272      0.        ]
 [343374.65423331 112804.08486139      0.        ]
 [343372.68965667 112797.4458678       0.        ]
 [343370.5375162  112791.06033709      0.        ]
 [343368.51265401 112786.95106953      0.        ]]
polygo

KeyboardInterrupt: 

In [7]:
import geopandas as gpd
from shapely import geometry as geom
from pathlib import Path

gpkg_path = Path("/cpfs01/shared/opendrivelab/sii/jingchanghui/d123_workspace/data/maps/nuscenes_singapore-onenorth.gpkg")
layers = ['lane', 'lane_group', 'intersection', 'crosswalk', 'walkway', 'carpark', 'generic_drivable']

def check_gpkg_layer(gpkg_path, layer_name, max_coords=5):
    gdf = gpd.read_file(gpkg_path, layer=layer_name)
    print(f"Layer: {layer_name}, rows: {len(gdf)}, geometry types: {gdf.geometry.type.unique()}")
    for i, geom_obj in enumerate(gdf.geometry[:5]):
        if geom_obj is None:
            continue
        if isinstance(geom_obj, geom.Polygon):
            print(f"  {i} exterior coords sample: {list(geom_obj.exterior.coords)[:max_coords]}")

for layer in layers:
    check_gpkg_layer(gpkg_path, layer)


Layer: lane, rows: 936, geometry types: ['Polygon']
  0 exterior coords sample: [(344.9027221434599, 1485.9780192382011), (340.55044611124424, 1501.8479583256944), (336.7942081997032, 1515.7482088513757), (330.7973034415235, 1537.0140791695299), (329.5577311749576, 1541.5753467907607)]
  1 exterior coords sample: [(345.7720232924004, 1410.0749168672085), (350.21929863187677, 1407.6543901915875), (350.60575211189615, 1408.0144636302223), (351.86847671379013, 1408.8312346340717), (353.0847839656397, 1410.03549479213)]
  2 exterior coords sample: [(853.081749678674, 1835.2890244679431), (858.6498815406115, 1841.9131413480395), (855.773081102668, 1847.43682472174), (855.7743569527995, 1847.4219085585241), (855.5129848557752, 1846.168079017607)]
  3 exterior coords sample: [(1117.9125737817687, 769.9483249283255), (1111.875905544529, 774.3630661080969), (1111.3641525583166, 774.3509460040419), (1109.3552524662064, 774.5647522860593), (1108.6703972503192, 774.4431133726303)]
  4 exterior coo

In [8]:
import geopandas as gpd
import numpy as np
import trimesh
from shapely.geometry import Polygon
from trimesh.creation import triangulate_polygon

# GPKG 文件路径
gpkg_path = "/cpfs01/shared/opendrivelab/sii/jingchanghui/d123_workspace/data/maps/nuscenes_singapore-onenorth.gpkg"

# 读取 lane 图层
layer_name = "lane"
gdf = gpd.read_file(gpkg_path, layer=layer_name)
print(f"{layer_name} 图层数量:", len(gdf))

# 取第一个 Polygon
poly = gdf.geometry.iloc[0]
print("Polygon 类型:", type(poly))
print("Polygon 面积:", poly.area)

# 生成 3D 顶点 (Z=0)
poly_3d = np.column_stack([np.array(poly.exterior.coords), np.zeros(len(poly.exterior.coords))])

# Triangulate 2D polygon
_, faces = triangulate_polygon(poly)
mesh = trimesh.Trimesh(vertices=poly_3d, faces=faces)

print("生成 Trimesh 顶点数量:", len(mesh.vertices))
print("生成 Trimesh 面片数量:", len(mesh.faces))


lane 图层数量: 936
Polygon 类型: <class 'shapely.geometry.polygon.Polygon'>
Polygon 面积: 194.2107756214646
生成 Trimesh 顶点数量: 11
生成 Trimesh 面片数量: 9


In [9]:
import fiona
from pathlib import Path

gpkg_path = Path("/cpfs01/shared/opendrivelab/sii/jingchanghui/d123_workspace/data/maps/nuscenes_singapore-onenorth.gpkg")

# 列出 gpkg 文件中的所有图层
layers = fiona.listlayers(gpkg_path)
print("GPKG layers:", layers)

# 读取其中一个图层，比如 "lane"
import geopandas as gpd
lane_gdf = gpd.read_file(gpkg_path, layer="lane")
print("Lane GeoDataFrame:", lane_gdf.shape)
print(lane_gdf.head())


GPKG layers: ['lane', 'lane_group', 'intersection', 'crosswalk', 'walkway', 'carpark', 'generic_drivable']
Lane GeoDataFrame: (936, 9)
                                     id lane_group_id  speed_limit_mps  \
0  000e869e-a0b4-4d71-99b7-6392c529d3bb          None              0.0   
1  0059e752-5800-44a5-9798-b1c97529f3c5          None              0.0   
2  009f41e9-56c1-467b-80b7-08851d132932          None              0.0   
3  0107439e-5e59-4238-b363-e43321cb86c9          None              0.0   
4  012246ce-c1e1-4eaf-9783-873488e5f2ea          None              0.0   

                                     predecessor_ids  \
0           ['e8b58454-b4c8-4d47-aed3-28af9bef06b5']   
1           ['1411cd2c-920b-44f5-a424-d932e7c3c1e5']   
2                                                 []   
3           ['ac3ce586-041a-47eb-b6f2-bf1ed3471085']   
4  ['6378690e-cb8d-4092-b8fc-573acbd2eb77', '940a...   

                              successor_ids left_boundary right_boundary  \
0  ['4f

In [4]:
import trimesh
from shapely.geometry import Polygon

geom = lane_gdf.geometry.iloc[0]

if isinstance(geom, Polygon):
    # 返回 (vertices, faces) tuple
    vertices, faces = trimesh.creation.triangulate_polygon(geom)
    print("mesh vertices:", len(vertices))
    print("mesh faces:", len(faces))
else:
    print("Not a polygon, type:", type(geom))


mesh vertices: 12
mesh faces: 9


In [5]:
import geopandas as gpd
from pathlib import Path
from shapely.geometry import Polygon, MultiPolygon

# 打开 gpkg
gpkg_path = Path("/cpfs01/shared/opendrivelab/sii/jingchanghui/d123_workspace/data/maps/nuscenes_singapore-onenorth.gpkg")

# 选择 mesh 候选图层
mesh_layers = ["carpark", "intersection", "walkway", "generic_drivable"]

for layer in mesh_layers:
    gdf = gpd.read_file(gpkg_path, layer=layer)
    n_polygons = gdf.geometry.apply(lambda g: isinstance(g, (Polygon, MultiPolygon))).sum()
    print(f"Layer {layer}: {len(gdf)} records, {n_polygons} polygons")
    if n_polygons > 0:
        print("  Example geometry:", gdf.geometry.iloc[0])


Layer carpark: 39 records, 39 polygons
  Example geometry: POLYGON ((342350.7500227472 113264.74096178917, 342355.4209799833 113266.36624312984, 342364.2951965948 113241.22875732943, 342379.42158557964 113197.81962831265, 342374.65416838485 113196.64304488104, 342359.509340056 113239.92284675498, 342350.7500227472 113264.74096178917))
Layer intersection: 643 records, 643 polygons
  Example geometry: POLYGON ((343176.4870034008 112911.15032486526, 343173.0026948686 112912.87630608182, 343166.0592637735 112904.05432028243, 343172.06881074666 112905.9294798103, 343175.7718760949 112903.59611497645, 343176.2125744334 112904.60004689802, 343176.6896325589 112905.86115492179, 343179.68739263155 112909.45114395097, 343176.4870034008 112911.15032486526))
Layer walkway: 838 records, 838 polygons
  Example geometry: POLYGON ((342841.22090017376 113462.94975837781, 342839.99676847854 113461.09532287538, 342838.69353662396 113471.18971419384, 342837.7452782714 113478.32482611825, 342837.2239277139

In [6]:
import geopandas as gpd
import fiona

gpkg_path = "/cpfs01/shared/opendrivelab/sii/jingchanghui/d123_workspace/data/maps/nuscenes_singapore-onenorth.gpkg"

# 列出所有图层
layers = fiona.listlayers(gpkg_path)
for layer in layers:
    gdf = gpd.read_file(gpkg_path, layer=layer)
    print(layer, gdf.geometry.type.unique())


lane ['Polygon']
lane_group ['Polygon']
intersection ['Polygon']
crosswalk ['Polygon']
walkway ['Polygon']
carpark ['Polygon']
generic_drivable ['Polygon']


In [7]:
import geopandas as gpd
import fiona

def inspect_polygons(gpkg_path, layers=None, head_n=3):
    """
    检查 GPKG 文件里各个图层的 Polygon 是否有面积（真 Polygon vs 伪 Polygon）
    
    参数:
        gpkg_path: str, gpkg 文件路径
        layers: list[str], 要检查的图层 (默认检查所有图层)
        head_n: int, 每层打印前多少个几何的详细信息
    """
    all_layers = fiona.listlayers(gpkg_path)
    if layers is None:
        layers = all_layers

    print(f"📂 文件: {gpkg_path}")
    print(f"🔍 总共有 {len(all_layers)} 个图层, 本次检查 {len(layers)} 个\n")

    for layer in layers:
        gdf = gpd.read_file(gpkg_path, layer=layer)
        geom_types = gdf.geometry.geom_type.value_counts()
        area_stats = gdf.geometry.area.describe()

        print(f"=== Layer: {layer} ===")
        print("几何类型统计:\n", geom_types)
        print("面积统计:\n", area_stats, "\n")

        for i, geom in enumerate(gdf.geometry.head(head_n)):
            print(f"  {i}: type={geom.geom_type}, valid={geom.is_valid}, area={geom.area}")
        print("-" * 40)

# 用法示例：
gpkg_path = "/cpfs01/shared/opendrivelab/sii/jingchanghui/d123_workspace/data/maps/nuscenes_singapore-onenorth.gpkg"
inspect_polygons(gpkg_path, layers=["lane", "lane_group", "walkway", "crosswalk"])


📂 文件: /cpfs01/shared/opendrivelab/sii/jingchanghui/d123_workspace/data/maps/nuscenes_singapore-onenorth.gpkg
🔍 总共有 7 个图层, 本次检查 4 个

=== Layer: lane ===
几何类型统计:
 Polygon    936
Name: count, dtype: int64
面积统计:
 count     936.000000
mean      137.795441
std       147.710841
min         6.650064
25%        46.608833
50%        91.323722
75%       166.914372
max      1519.032449
dtype: float64 

  0: type=Polygon, valid=True, area=194.1709649261919
  1: type=Polygon, valid=True, area=25.822912923429023
  2: type=Polygon, valid=True, area=42.87517603571976
----------------------------------------
=== Layer: lane_group ===
几何类型统计:
 Polygon    781
Name: count, dtype: int64
面积统计:
 count     781.000000
mean      238.544517
std       284.973673
min        15.793930
25%        71.518830
50%       134.468442
75%       314.197114
max      3092.092476
dtype: float64 

  0: type=Polygon, valid=True, area=120.35292153925182
  1: type=Polygon, valid=True, area=186.32940200233222
  2: type=Polygon, valid

In [22]:
import trimesh
geom = gdf.geometry.iloc[0]
vertices, faces = trimesh.creation.triangulate_polygon(geom)
mesh = trimesh.Trimesh(vertices=vertices, faces=faces)
print("vertices:", len(mesh.vertices), "faces:", len(mesh.faces))


vertices: 21 faces: 19


In [13]:
import geopandas as gpd
import numpy as np

gpkg_path = "/cpfs01/shared/opendrivelab/sii/jingchanghui/d123_workspace/data/maps/nuscenes_singapore-onenorth.gpkg"
gdf = gpd.read_file(gpkg_path, layer="lane")

# 所有 Polygon 的所有坐标
coords = np.vstack([np.array(p.exterior.coords) for p in gdf.geometry])
center = coords.mean(axis=0)
print("地图中心坐标:", center)


地图中心坐标: [342803.45164393 113608.0080534 ]


In [14]:
from pathlib import Path
from d123.dataset.scene.arrow_scene import ArrowScene
scene_path = Path("/cpfs01/shared/opendrivelab/sii/jingchanghui/d123_workspace/data/train/9a61a88ed9094334a73aa93c08222110.arrow")
scene = ArrowScene(scene_path)
map_api = scene.map_api 
map_path = getattr(map_api, "_file_path", None)
map_name = map_path.stem
map_prefix = map_name.split("_")[0]
print("Map file path:", map_name)
print("Map file path:", map_prefix)
print(type(map_api).__name__)
if map_prefix == "nuscenes":
    print(1)
    
from d123.dataset.maps.map_datatypes import MapLayer
map_layers = [
        MapLayer.LANE_GROUP,
        MapLayer.LANE,
        MapLayer.WALKWAY,
        MapLayer.CROSSWALK,
        MapLayer.CARPARK,
        MapLayer.GENERIC_DRIVABLE,
]
from typing import Final
MAP_RADIUS: Final[float] = 1000000
initial_ego_vehicle_state = scene.get_ego_state_at_iteration(0)
center = initial_ego_vehicle_state.center_se3
map_objects_dict = scene.map_api.get_proximal_map_objects(center.point_2d, radius=MAP_RADIUS, layers=map_layers)
print(map_objects_dict)

Map file path: nuscenes_singapore-onenorth
Map file path: nuscenes
GPKGMap
1
defaultdict(<class 'list'>, {<MapLayer.LANE_GROUP: 1>: [<d123.dataset.maps.gpkg.gpkg_map_objects.GPKGLaneGroup object at 0x7f49ee4fdf70>, <d123.dataset.maps.gpkg.gpkg_map_objects.GPKGLaneGroup object at 0x7f49ea9362d0>, <d123.dataset.maps.gpkg.gpkg_map_objects.GPKGLaneGroup object at 0x7f49ea935580>, <d123.dataset.maps.gpkg.gpkg_map_objects.GPKGLaneGroup object at 0x7f49ea935ca0>, <d123.dataset.maps.gpkg.gpkg_map_objects.GPKGLaneGroup object at 0x7f49ea935850>, <d123.dataset.maps.gpkg.gpkg_map_objects.GPKGLaneGroup object at 0x7f49ea935430>, <d123.dataset.maps.gpkg.gpkg_map_objects.GPKGLaneGroup object at 0x7f49ea935970>, <d123.dataset.maps.gpkg.gpkg_map_objects.GPKGLaneGroup object at 0x7f49ea9357f0>, <d123.dataset.maps.gpkg.gpkg_map_objects.GPKGLaneGroup object at 0x7f49ea935c40>, <d123.dataset.maps.gpkg.gpkg_map_objects.GPKGLaneGroup object at 0x7f49ea936270>, <d123.dataset.maps.gpkg.gpkg_map_objects.GPKGLa

In [8]:
import trimesh
import shapely.geometry as geom
import numpy as np

# 构造一个简单的矩形 polygon
poly = geom.Polygon([(0,0), (1,0), (1,1), (0,1)])

# 调用 trimesh 的三角化
result = trimesh.creation.triangulate_polygon(poly)

print("返回类型:", type(result))
print("返回内容:", result)

# 如果是 list，每个元素应该是一个 shapely.Polygon
if isinstance(result, list):
    for i, tri in enumerate(result):
        print(f"triangle {i}:", tri, "coords:", np.array(tri.exterior.coords))


返回类型: <class 'tuple'>
返回内容: (array([[0., 0.],
       [1., 0.],
       [1., 1.],
       [0., 1.],
       [0., 0.]]), array([[3, 0, 1],
       [1, 2, 3]]))


In [12]:
# 导入必要模块
from pathlib import Path
from d123.dataset.scene.arrow_scene import ArrowScene

# 替换成你自己的场景文件路径
scene_path = Path("/cpfs01/shared/opendrivelab/sii/jingchanghui/d123_workspace/data/train/9a61a88ed9094334a73aa93c08222110.arrow")

# 加载场景
scene = ArrowScene(scene_path)

# 获取初始 ego 车状态
initial_ego_state = scene.get_ego_state_at_iteration(0)
center_se3 = initial_ego_state.center_se3  # StateSE3 类型

# 打印全局坐标
print("Ego 初始中心坐标 (全局坐标系):")
print(f"X: {center_se3.x}")
print(f"Y: {center_se3.y}")
print(f"Z: {center_se3.z}")

# 如果想要 Point3D 形式
print("\nPoint3D 格式:")
print(center_se3.point_3d)


Ego 初始中心坐标 (全局坐标系):
X: 852.7883802515286
Y: 1652.293697739046
Z: 0.0

Point3D 格式:
Point3D(x=852.7883802515286, y=1652.293697739046, z=0.0)


In [13]:
# 导入必要模块
from pathlib import Path
from d123.dataset.scene.arrow_scene import ArrowScene

# 替换成你自己的场景文件路径
scene_path = Path("/cpfs01/shared/opendrivelab/sii/jingchanghui/d123_workspace/data/nuplan_mini_train/2021.05.12.23.36.44_veh-35_00152_00504.arrow")
# 加载场景
scene = ArrowScene(scene_path)

# 获取初始 ego 车状态
initial_ego_state = scene.get_ego_state_at_iteration(0)
center_se3 = initial_ego_state.center_se3  # StateSE3 类型

# 打印全局坐标
print("Ego 初始中心坐标 (全局坐标系):")
print(f"X: {center_se3.x}")
print(f"Y: {center_se3.y}")
print(f"Z: {center_se3.z}")

# 如果想要 Point3D 形式
print("\nPoint3D 格式:")
print(center_se3.point_3d)


Ego 初始中心坐标 (全局坐标系):
X: 664480.8837864517
Y: 3996649.089082208
Z: 621.5858883665154

Point3D 格式:
Point3D(x=664480.8837864517, y=3996649.089082208, z=621.5858883665154)


In [2]:
    import os
    import subprocess
    import argparse

    # 定义数据集下载URL列表（根据需要选择要下载的文件）
    NUSCENES_URLS = {
        # 完整数据集（1.4TB）
        "v1.0-trainval01_blobs": "https://s3.amazonaws.com/nuscenes.org/data/v1.0-trainval01_blobs.tgz",
        "v1.0-trainval02_blobs": "https://s3.amazonaws.com/nuscenes.org/data/v1.0-trainval02_blobs.tgz",
        "v1.0-trainval03_blobs": "https://s3.amazonaws.com/nuscenes.org/data/v1.0-trainval03_blobs.tgz",
        "v1.0-trainval04_blobs": "https://s3.amazonaws.com/nuscenes.org/data/v1.0-trainval04_blobs.tgz",
        "v1.0-trainval05_blobs": "https://s3.amazonaws.com/nuscenes.org/data/v1.0-trainval05_blobs.tgz",
        "v1.0-trainval06_blobs": "https://s3.amazonaws.com/nuscenes.org/data/v1.0-trainval06_blobs.tgz",
        "v1.0-trainval07_blobs": "https://s3.amazonaws.com/nuscenes.org/data/v1.0-trainval07_blobs.tgz",
        "v1.0-trainval08_blobs": "https://s3.amazonaws.com/nuscenes.org/data/v1.0-trainval08_blobs.tgz",
        "v1.0-trainval09_blobs": "https://s3.amazonaws.com/nuscenes.org/data/v1.0-trainval09_blobs.tgz",
        "v1.0-trainval10_blobs": "https://s3.amazonaws.com/nuscenes.org/data/v1.0-trainval10_blobs.tgz",
        
        # 元数据（1.5GB）
        "v1.0-trainval_meta": "https://s3.amazonaws.com/nuscenes.org/data/v1.0-trainval_meta.tgz",
        
        # 其他可选数据集
        "v1.0-test_blobs": "https://s3.amazonaws.com/nuscenes.org/data/v1.0-test_blobs.tgz",
        "v1.0-test_meta": "https://s3.amazonaws.com/nuscenes.org/data/v1.0-test_meta.tgz",
        "v1.0-mini": "https://s3.amazonaws.com/nuscenes.org/data/v1.0-mini.tgz",  # 迷你版（1.4GB）
    }

    def check_dependencies():
        """检查是否安装了必要的依赖工具"""
        try:
            # 检查wget是否可用
            subprocess.run(["wget", "--version"], check=True, 
                           stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            # 检查tar是否可用
            subprocess.run(["tar", "--version"], check=True, 
                           stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            return True
        except FileNotFoundError as e:
            print(f"错误: 缺少必要的工具: {e}")
            print("请安装wget和tar后再试")
            return False

    def download_file(url, save_path):
        """使用wget下载文件"""
        print(f"开始下载: {url}")
        try:
            # 下载文件，显示进度条
            subprocess.run(["wget", "-c", "-P", save_path, url], check=True)
            print(f"下载完成: {os.path.basename(url)}")
            return True
        except subprocess.CalledProcessError as e:
            print(f"下载失败: {e}")
            return False

    def extract_file(file_path, extract_path):
        """解压tar文件"""
        print(f"开始解压: {file_path}")
        try:
            subprocess.run(["tar", "-xzf", file_path, "-C", extract_path], check=True)
            print(f"解压完成: {file_path}")
            return True
        except subprocess.CalledProcessError as e:
            print(f"解压失败: {e}")
            return False

    def main():
        # 设置命令行参数
        parser = argparse.ArgumentParser(description='下载NuScenes数据集')
        parser.add_argument('--dataset', nargs='+', 
                           choices=NUSCENES_URLS.keys(),
                           default=['v1.0-mini'],  # 默认下载迷你版
                           help='指定要下载的数据集部分')
        parser.add_argument('--no-extract', action='store_true',
                           help='只下载不解压')
        args = parser.parse_args()

        # 检查依赖
        if not check_dependencies():
            return

        # 目标下载路径
        target_path = "/inspire/hdd/project/roboticsystem2/public/nuscenes"
        
        # 创建目标目录（如果不存在）
        os.makedirs(target_path, exist_ok=True)
        print(f"数据集将下载到: {target_path}")

        # 下载并解压指定的数据集部分
        for dataset in args.dataset:
            url = NUSCENES_URLS[dataset]
            filename = os.path.basename(url)
            file_path = os.path.join(target_path, filename)

            # 检查文件是否已下载
            if os.path.exists(file_path):
                print(f"文件已存在: {filename}，跳过下载")
            else:
                # 下载文件
                if not download_file(url, target_path):
                    print(f"跳过 {dataset} 的处理")
                    continue

            # 如果不需要解压，或者已经解压过，就跳过
            if not args.no_extract:
                extract_dir = os.path.splitext(filename)[0]
                if os.path.exists(os.path.join(target_path, extract_dir)):
                    print(f"文件已解压: {extract_dir}，跳过解压")
                else:
                    extract_file(file_path, target_path)

        print("所有操作完成")

    if __name__ == "__main__":
        main()
    

usage: ipykernel_launcher.py [-h]
                             [--dataset {v1.0-trainval01_blobs,v1.0-trainval02_blobs,v1.0-trainval03_blobs,v1.0-trainval04_blobs,v1.0-trainval05_blobs,v1.0-trainval06_blobs,v1.0-trainval07_blobs,v1.0-trainval08_blobs,v1.0-trainval09_blobs,v1.0-trainval10_blobs,v1.0-trainval_meta,v1.0-test_blobs,v1.0-test_meta,v1.0-mini} [{v1.0-trainval01_blobs,v1.0-trainval02_blobs,v1.0-trainval03_blobs,v1.0-trainval04_blobs,v1.0-trainval05_blobs,v1.0-trainval06_blobs,v1.0-trainval07_blobs,v1.0-trainval08_blobs,v1.0-trainval09_blobs,v1.0-trainval10_blobs,v1.0-trainval_meta,v1.0-test_blobs,v1.0-test_meta,v1.0-mini} ...]]
                             [--no-extract]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9003 --control=9001 --hb=9000 --Session.signature_scheme="hmac-sha256" --Session.key=b"df6d227b-777b-4e41-a233-bfadaecccf93" --shell=9002 --transport="tcp" --iopub=9004 --f=/root/.local/share/jupyter/runtime/kernel-v2-1420357eHnnVFNOe

SystemExit: 2

/inspire/hdd/project/roboticsystem2/jingzhanghui-253108140204/env/d123/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
